In [1]:
!pip install lumibot==2.9.13

In [1]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta
from finbert_utils import estimate_sentiment

# Importing Essential Libraries and Classes

In this cell, we import the necessary libraries and classes to set up our trading environment and sentiment analysis capabilities:

- **Alpaca**: A brokerage class from `lumibot.brokers` to interact with the Alpaca trading platform.
- **YahooDataBacktesting**: A class from `lumibot.backtesting` for obtaining historical data from Yahoo Finance for backtesting purposes.
- **Strategy**: The base strategy class from `lumibot.strategies` that we will inherit to define our custom trading strategy.
- **Trader**: A class from `lumibot.traders` to execute trades based on the signals generated by our strategy.
- **datetime**: From the built-in `datetime` module, for handling dates and times, crucial for timestamping trades and managing historical data.
- **REST**: Imported from `alpaca_trade_api`, providing REST API access to the Alpaca platform for trading actions and data retrieval.
- **Timedelta**:  is used for calculating time differences.
- **estimate_sentiment**: A function from `finbert_utils` for estimating the sentiment of financial texts, which can influence trading decisions.

This setup is foundational for creating a trading strategy that utilizes sentiment analysis and can execute trades automatically through the Alpaca platform, with backtesting capabilities using Yahoo Finance data.


In [2]:
API_KEY="Your api key here"
API_SECRET="Your api key here"
BASE_URL="https://paper-api.alpaca.markets/v2"
ALPACA_CREDS={
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER":True
}


# Configuration of Alpaca API Credentials

Here, we establish the API credentials necessary for connecting to the Alpaca trading platform. This involves specifying our API key, API secret, and the base URL for Alpaca's paper trading API. These credentials are then organized into a dictionary named `ALPACA_CREDS` which will be used throughout the project to authenticate our requests to Alpaca. Paper trading mode is enabled (`"PAPER": True`), allowing us to test our trading strategies in a simulated environment without financial risk.




In [3]:

class MLTrader(Strategy):
    def initialize(self,symbol:str="SPY",cash_at_risk:float=0.5):
        self.symbol=symbol
        self.sleeptime="24H"
        self.last_trade=None
        self.cash_at_risk=cash_at_risk
        self.api=REST(base_url=BASE_URL,key_id=API_KEY,secret_key=API_SECRET)
        
    def position_sizing(self):
        cash=self.get_cash()
        last_price=self.get_last_price(self.symbol)
        quantity=round(cash*self.cash_at_risk/last_price,0)
        return cash,last_price,quantity
    
#This formula guides how much of our cash balance we use per trade. cash_at_risk of 0.5 menas that for each trade we are using 50% of our remaining cash balance
    def get_dates(self):
        today=self.get_datetime()
        three_days_prior=today-Timedelta(days=3)
        return today.strftime('%Y-%m-%d'),three_days_prior.strftime('%Y-%m-%d')
    
    def get_sentiment(self):
        today,three_days_prior =self.get_dates()
        news=self.api.get_news(symbol=self.symbol,
                                start=three_days_prior,
                                end=today)
        news=[ev.__dict__["_raw"]["headline"] for ev in news]
        probability, sentiment=estimate_sentiment(news)
        return probability, sentiment, news 
        
            
    def on_trading_iteration(self):
        cash,last_price,quantity=self.position_sizing()
        probability, sentiment, news=self.get_sentiment()
        if cash>last_price:
            if sentiment=="positive" and probability> 0.999:
                #print(probability, sentiment, news)
                if self.last_trade=="sell":
                    self.sell_all()
                order=self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="market",
                    take_profit_price=last_price*1.20,
                    stop_loss_price=last_price*0.95,
                    )
                self.submit_order(order)
                self.last_trade="buy"
            
            elif sentiment=="negative" and probability> 0.999:
                #print(probability, sentiment, news)
                if self.last_trade=="buy":
                    self.sell_all()
                order=self.create_order(
                    self.symbol,
                    quantity,
                    "sell",
                    type="market",
                    take_profit_price=last_price*0.8,
                    stop_loss_price=last_price*1.05,
                    )
                self.submit_order(order)
                self.last_trade="sell"
            
start_date=datetime(2020,1,1)
end_date=datetime(2024,4,5)
broker=Alpaca(ALPACA_CREDS)

strategy=MLTrader(name="mlstrat",
                    broker=broker,
                    parameters={"symbol":"SPY",
                            "cash_at_risk":0.5}
                )

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY","cash_at_risk":0.5}
)

Starting backtest for MLTrader...


INFO:backtest_stats:Starting backtest...


Progress || 100.00%  [Elapsed: 0:15:09 ETA: 0:00:00] Portfolio Val: 562,140.67
Creating trades plot...

Creating indicators plot...

Creating tearsheet...


{'cagr': 0.5009147055138508,
 'volatility': 0.5769104744862977,
 'sharpe': 0.7778758123391128,
 'max_drawdown': {'drawdown': 0.6475489346357906,
  'date': Timestamp('2020-05-13 09:30:00-0400', tz='America/New_York')},
 'romad': 0.7735549836022614,
 'total_return': 4.621406662780757}

# MLTrader Strategy Definition and Backtest Setup

This cell defines `MLTrader`, a custom trading strategy class inheriting from `lumibot`'s `Strategy`. It's designed to trade based on sentiment analysis, with specific attention to managing risk and optimizing trade positions. Here's an overview of its key components:

1. **Initialization (`initialize`)**: Sets up the trading symbol (e.g., SPY), the amount of cash at risk per trade, and initializes the Alpaca API for trading actions.
2. **Position Sizing (`position_sizing`)**: Determines the amount of stock to buy or sell based on the available cash and a predefined risk parameter.
3. **Sentiment Analysis (`get_sentiment`)**: Fetches recent news headlines for the trading symbol and estimates their sentiment, guiding the trading decisions.
4. **Trading Logic (`on_trading_iteration`)**: Executes buy or sell orders based on sentiment analysis, with conditions for entry and exit, including take profit and stop loss levels.

## Visual Representation

- **Position Sizing Formula**: `quantity = round((cash * cash_at_risk) / last_price)`
- **Trading Decision Flowchart**:

![Trading Decision Flowchart](flowchart.png)


This flowchart should depict how the trading strategy decides when to enter or exit trades based on the sentiment analysis and predefined thresholds for action.

## Backtesting the Strategy

Finally, we set up a backtest for this strategy using historical data from Yahoo Finance, spanning from January 1, 2020, to April 5, 2024. The backtest uses the Alpaca broker for simulated trades within this period.


